# Mixed Integer Programming

The following sections present an example of a MIP problem and show how to solve it. Here's the problem:

__Maximize $x + 10y$__ subject to the following constraints:

$x + 7 y	≤	17.5$

$x	≤	3.5$

$x	≥	0$

$y	≥	0$

$x, y$ are integers

Since the constraints are linear, this is just a linear optimization problem in which the solutions are required to be integers. The graph below shows the integer points in the feasible region for the problem.


<img src="https://developers.google.com/optimization/images/mip/feasible_region.png" width="700">

Notice that this problem is very similar to the linear optimization problem, but in this case we require the solutions to be integers.



##Using Scipy for mixed integer linear programing 

Check help: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.milp.html


In [48]:
import numpy as np
from scipy.optimize import milp
from scipy.optimize import LinearConstraint



Since milp requires that the problem be expressed as a minimization problem, the objective function coefficients on the decision variables are as below.
Note the negative sign: we maximize the original objective function by minimizing the negative of the objective function.

In [49]:
c = -np.array([1, 10])

We collect the coefficients of the constraints into arrays like:

In [50]:
A = np.array([[1, 7], [1, 0]])
b_u = np.array([17.5, 3.5])
b_l = np.full_like(b_u, -np.inf)   # You can use  np.array([-np.inf, -np.inf])

In [51]:
print(f'\n A:')
print(A , end = '\n')
print(f'\n b_u:')
print(b_u , end = '\n')
print(f'\n b_l:')
print(b_l , end = '\n')



 A:
[[1 7]
 [1 0]]

 b_u:
[17.5  3.5]

 b_l:
[-inf -inf]


These arrays are collected into a single `LinearConstraint` object like:

In [52]:
constraints = LinearConstraint(A, b_l, b_u)

Finally, the problem states that both decision variables must be integers:

In [53]:
integrality = np.ones_like(c)

In [54]:
integrality

array([1, 1])

We solve the problem like:

In [55]:
res = milp(c=c, constraints=constraints, integrality=integrality)
res.x

array([3., 2.])

## Comparing Linear and Integer Optimization
Let's compare the solution to the integer optimization problem, shown above, with the solution to the corresponding linear optimization problem, in which integer constraints are removed. You might guess that the solution to the integer problem would be the integer point in the feasible region closest to the linear solution — namely, the point $x = 0,  y = 2$. But as you will see next, this is not the case.



You can solve the relaxed problem without integrality constraints. Or you can use `linprog` function the same privious tutorials.

In [56]:
res = milp(c=c, constraints=constraints)  # OR:
# from scipy.optimize import linprog; res = linprog(c, A, b_u)
res.x

array([0. , 2.5])

The solution to the linear problem occurs at the point x = 0,  y = 2.5, where the objective function equals 25. Here's a graph showing the solutions to both the linear and integer problems.


<img src="https://developers.google.com/optimization/images/mip/feasible_region_sol.png" width="700">

Notice that the integer solution is not close to the linear solution, compared with most other integer points in the feasible region. In general, the solutions to a linear optimization problem and the corresponding integer optimization problems can be far apart. Because of this, the two types of problems require different methods for their solution.


# Non-linear Programming



## Nonlinear Programming

<img src="https://www.researchgate.net/profile/Tamas-Terlaky/publication/260790215/figure/fig1/AS:614092289097741@1523422419670/Example-of-a-nonlinear-optimization-problem-with-the-humpback-function-as-objective_W640.jpg" width="700">

For nonlinear optimization there are many available packages in Python. In this tutorial, you will learn how to use **Scipy** pakcage to solve the following problem:

**Minimize: $ x_1 x_4 (x_1 + x_2 + x_3) + x_3$**

*Subject to following constraints:*

$x_1 x_2 x_3 x_4 \ge 25$

$x_1^2 + x_2^2 + x_3^2 + x_4^2 = 40$

$1 \le x_1 , x_2 , x_3 , x_4 \le 5$

*Initial Guess:*

$x_0 = (1,5,5,1)$


### Using Scipy

In [57]:
import numpy as np
from scipy.optimize import minimize

def objective(x):
    return x[0]*x[3]*(x[0]+x[1]+x[2])+x[2]

def constraint1(x):
    return x[0]*x[1]*x[2]*x[3]-25.0

def constraint2(x):
    sum_eq = 40.0
    for i in range(4):
        sum_eq = sum_eq - x[i]**2
    return sum_eq

# initial guesses
n = 4
x0 = np.zeros(n)
x0[0] = 1.0
x0[1] = 5.0
x0[2] = 5.0
x0[3] = 1.0

# show initial objective
print('Initial SSE Objective: ' + str(objective(x0)))

# optimize
b = (1.0,5.0)
bnds = (b, b, b, b)
con1 = {'type': 'ineq', 'fun': constraint1}
con2 = {'type': 'eq', 'fun': constraint2}
cons = ([con1,con2])
solution = minimize(objective,x0,method='SLSQP',\
                    bounds=bnds,constraints=cons)
x = solution.x

# show final objective
print('Final SSE Objective: ' + str(objective(x)))

# print solution
print('Solution')
print('x1 = ' + str(x[0]))
print('x2 = ' + str(x[1]))
print('x3 = ' + str(x[2]))
print('x4 = ' + str(x[3]))

Initial SSE Objective: 16.0
Final SSE Objective: 17.01401724563517
Solution
x1 = 1.0
x2 = 4.742996096883977
x3 = 3.8211546234095715
x4 = 1.379407645075325
